In [1]:
import os, glob, re

import pandas as pd
import numpy as np

from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
from transferlearning import ConvBaseSearch

# Glob all jpg-ending files and regex the category out of each
p = Path("data/*/*")
paths_images = glob.glob(str(p) + "*jpg")
cat = [re.search("\d+\.", i).group().strip(".") for i in paths_images]

df = pd.DataFrame(list(zip(paths_images,cat)), columns=["path", "cat"])
df = df[df['cat'].map(df['cat'].value_counts()) > 150]

target_col, path_col = "cat", "path"

pretrained_models = ["vgg16", "vgg19"] #Models that should be tested

Using TensorFlow backend.


In [3]:
# Initialize ConvBaseSearch and pass the model names you want to test
cbs = ConvBaseSearch(pretrained_models)

In [6]:
import torch
print(torch.cuda.is_available())

True


In [7]:
# -*- coding: utf-8 -*-
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 30918460.297300883
1 26816114.09336842
2 26627740.458248965
3 25654347.267109454
4 21875774.895298693
5 15734826.634910446
6 9802976.45263368
7 5600789.419625288
8 3199924.333483309
9 1945893.0272589237
10 1301375.3442368507
11 952060.2376855612
12 745278.6015071997
13 609747.140243778
14 512805.1632633341
15 438639.01064992853
16 379369.1741590543
17 330751.010252889
18 290187.5856081849
19 255905.86954074336
20 226653.00788524354
21 201530.52875877568
22 179807.63179573725
23 160925.2470097556
24 144455.93778561143
25 130025.55628528679
26 117354.22441616254
27 106169.89942296047
28 96262.75753540755
29 87455.70628169496
30 79608.69174316981
31 72595.03979501214
32 66309.07506801258
33 60660.7385308112
34 55573.46280716207
35 50984.081296372155
36 46836.66543094514
37 43080.78981689315
38 39674.433072582324
39 36583.654917995416
40 33769.9112940329
41 31204.70322255723
42 28862.226316518376
43 26720.892459806935
44 24761.116038107426
45 22966.80127163222
46 21320.605035476754
47 19

In [8]:
import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 749.316650390625
199 5.224360942840576
299 0.0619528591632843
399 0.0012128647649660707
499 0.00011697161971824244


In [4]:
# Fit the data to the pretrained models. Only use sample_size proportion of whole dataset.
cbs.fit(df, path_col, target_col, sample_size=0.02, data_augmentation=True)

Found 163 validated image filenames belonging to 12 classes.
Found 41 validated image filenames belonging to 12 classes.
Fitting  vgg16
Epoch 1/1


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node vgg16/block1_conv1/convolution (defined at /home/d4ve/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1617]

Function call stack:
keras_scratch_graph


In [5]:
# Get the best score 
cbs.best_score

0.2222222238779068

In [6]:
# Get the best model
best_base = cbs.best_base
print("Name:", best_base.name)
print("Keras model:", best_base.model)

Name: vgg16
Keras model: <keras.engine.training.Model object at 0x000002C074E10F88>
